In [ ]:
# This script calls bstar_deblaze.ipynb to get the normalized, deblazed spectrum for each star. 
# NOTE: bstar_deblaze currently calcalates but DOES NOT output the registered wavelength scale. 
#       you must first add that to the return statement before running this script (and make sure to remove it again
#       afterwords to use in smemp.ipynb, which does not require that output)

In [1]:
import import_ipynb 
import numpy as np
import astropy.io.fits as pf
from astropy.io import fits
from bstar_deblaze import bstar_deblazed
import os

importing Jupyter notebook from bstar_deblaze.ipynb


In [ ]:
path_to_dir = '/mnt_blpd0/datax/apf'
filelist = sorted(os.listdir(path_to_dir))
filelist.remove('apf_log.txt')
filelist.remove('apf_log_current.txt')
filelist.remove('apf_log_current_new.txt')
filelist.remove('apf_log_current_25mar2019.txt')
#filelist = ['ramq.233.fits']

bad_files = []
for file_name in filelist: 
    if not(file_name.startswith('ucb')):        
        try:
            # deal with directories
            if file_name.startswith('HIP') or os.path.isdir('./'+ file_name): # is a directory of spectra
                path = ('./' + file_name)
                spectra = sorted(os.listdir(path))
                image = np.zeros((79, 4608))
                for spectrum in spectra:
                    file = pf.open(path + '/' + spectrum)
                    data_part = file[0].data
                    if (str(np.shape(data_part)) != '(79, 4608)'):
                        print(str(np.shape(data_part)) + ' is an incompatible data shape.')
                        print('Cannot perform shift-and-match process.')
                        sys.exit()
                    image += data_part
            else: # just one spectrum
                file = pf.open(path_to_dir + '/' + file_name)
                old_header = file[0].header
                image = file[0].data

            orders  = np.linspace(30,51,22).astype(int)
            norm_reg_deblazed = np.zeros([len(orders), 4600])
            wl_shifted = np.zeros([len(orders), 4600])
            for order in orders:
                norm_reg_deb, shifted_wl_scale = bstar_deblazed(file, order)
                norm_reg_deblazed[order-30] = norm_reg_deb
                wl_shifted[order-30] = shifted_wl_scale

            new_header = old_header
            new_header.set('NRD', 'YES','Normalized, registered, deblazed')
            data_hdu = fits.PrimaryHDU(norm_reg_deblazed, new_header)
            wl_hdu = fits.ImageHDU(wl_shifted)
            hdu = fits.HDUList([data_hdu, wl_hdu])
            hdu.writeto('APF_spectra/Norm_Reg_Deb/' + file_name)
            print('Done: ' + file_name)
        except:
            print('Error occured processing file: ' + file_name)
            bad_files = bad_files + [file_name]
        
    # could either shift all the spectra by amount that scale is shifted and hstack all the spectra, 
    # or hstack all the shifted scales


In [ ]:
#TESTING BELOW HERE

In [49]:
test_fits = pf.open('APF_spectra/Norm_Reg_Deb/ramq.233.fits')
test_fits.info()
test_fits[0].header['NRD']


Filename: APF_spectra/Norm_Reg_Deb/ramq.233.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     229   (4600, 22)   float64   
  1                1 ImageHDU         7   (4600, 22)   float64   


'YES'